Miłosz Błachowiak

Porządkujemy dane zgodnie z protokołem TIER i zasadami 'tidy data'

In [1]:
import pandas as pd

# wczytanie danych z pliku 'weather_parsed.txt'
weather = pd.read_csv('..\Analysis Data\weather_parsed.txt',sep=';', header=None)

In [2]:
import pandas as pd

# wczytanie danych z pliku 'weather_parsed.txt'
weather = pd.read_csv('..\Analysis Data\weather_parsed.txt',sep=';', header=None)

In [3]:
# funkcja odpowiednio modyfikująca ciąg znaków, będzie użyta do rozdzielenia danych z pierwszej kolumny
def col_split(string):
    string = string[:11] + "_" + string[11:15] + "." + string[15:17] + "_" + string[17:]
    return string

In [4]:
# wywołanie funkcji col_split dla całej pierwszej kolumny
weather[0] = weather[0].apply(col_split)

In [5]:
# wstawienie nowych kolumn i rozdzielenie danych z pierwszej kolumny
weather.insert(1, "ID", "")
weather.insert(2, "Year and Month", "")
weather.insert(3, "Variable", "")


weather[['ID', 'Year and Month', 'Variable']] = weather[0].str.split('_', expand=True)

In [6]:
# usunięcie zbędnej kolumny
weather = weather.drop(0, axis=1)

In [7]:
# oczyszczenie i uporządkowanie danych
weather.replace([-9999], '-', inplace=True)

weather['Variable'] = weather['Variable'].str.lower()

In [8]:
# użycie funkcji melt - zamiana szerokiej tabeli na długą
weather = pd.melt(weather, id_vars=["ID", "Year and Month", "Variable"])

In [9]:
# połączenie kolumn "Year and Month"(rok i miesiąc) z kolumną "variable"(zawierającą dni miesiąca)
weather.insert(2, "Date", "")
weather["Date"] = weather["Year and Month"].astype(str) + "." +  weather["variable"].astype(str)

weather = weather.drop(["Year and Month", "variable"], axis=1)

In [10]:
# usunięcie wierszy nie posiadających danych pomiarowych
weather = weather[weather["value"] != '-']

In [11]:
# konwersja kolumny "Date" do formatu datetime
weather["Date"] = pd.to_datetime(weather["Date"])

In [12]:
# ustawienie indeksu dataframe'u na kolumnę "ID"
weather = weather.set_index("ID")

In [13]:
# posortowanie danych względem daty
weather["Date"] = weather["Date"].sort_values()


In [14]:
weather

,Date,Variable,value
ID,,,
MX000017004,1955-04-01,tmax,310
MX000017004,1955-04-01,tmin,150
MX000017004,1955-04-01,prcp,0
MX000017004,1955-04-02,tmax,310
MX000017004,1955-04-02,tmin,200
...,...,...,...
MX000017004,2011-04-02,prcp,0
MX000017004,2011-04-02,prcp,0
MX000017004,2011-04-02,tmax,365


In [15]:
# wyeksportowanie uporządkowanych danych do pliku csv

weather.to_csv('..\Analysis Data\weather_cleaned.csv')